In [2]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from precision_recall_cutoff import precision_recall_cutoff

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE, RFECV

#defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

#defining the file to be read from s3 bucket
file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datafile
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
#changing sales and salary to dummies
turnover = pd.concat([turnover.drop(columns=['sales','salary'],axis=1), pd.get_dummies(turnover[['sales','salary']])],axis=1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
#creating new features from InClass_10
turnover['interaction_1'] = np.where((turnover['satisfaction_level'] <= 0.465) & 
                                     (turnover['number_project'] > 2.5) & 
                                     (turnover['satisfaction_level'] >= 0.115), 1, 0)

turnover['interaction_2'] = np.where((turnover['satisfaction_level'] <= 0.465) & 
                                     (turnover['number_project'] <= 2.5) & 
                                     (turnover['last_evaluation'] <= 0.575), 1, 0)

turnover['interaction_3'] = np.where((turnover['satisfaction_level'] > 0.465) & 
                                     (turnover['time_spend_company'] <= 4.5) & 
                                     (turnover['average_montly_hours'] <= 290.5), 1, 0)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,interaction_1,interaction_2,interaction_3
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0


In [5]:
#defining input and target
x=turnover[['satisfaction_level', 'number_project', 'time_spend_company', 'interaction_1', 'interaction_3']]
y=turnover['left']

#splitting the data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y)

**GridSearchCV with Random Forest**

In [6]:
#defining parameter dictionary
ada_param_grid = {'n_estimators': [100, 300],
                  'estimator__min_samples_split': [10, 15],
                  'estimator__min_samples_leaf': [5, 7],
                  'estimator__max_depth': [3, 5, 7],
                  'learning_rate': [0.01]}

#running GridSearchCV
ada_grid_search = GridSearchCV(estimator = AdaBoostClassifier(estimator = DecisionTreeClassifier()), 
                               param_grid = ada_param_grid, 
                               cv = 3, 
                               scoring = 'f1',
                               n_jobs = -1).fit(x_train, y_train)

#extracting best hyperparameter combination
ada_grid_search.best_params_

{'estimator__max_depth': 7,
 'estimator__min_samples_leaf': 5,
 'estimator__min_samples_split': 10,
 'learning_rate': 0.01,
 'n_estimators': 300}

In [8]:
#building the model with best hyperparameters
ada_md = AdaBoostClassifier(estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 7),
                            n_estimators = 300, learning_rate = 0.01).fit(x_train, y_train)

#predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
ada_label = precision_recall_cutoff(y_test, ada_pred)

#print classification report
print(classification_report(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2286
           1       0.94      0.93      0.94       714

    accuracy                           0.97      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000



**RandomizedSearchCV with AdaBoost**

In [10]:
#running RandomizedSearchCV
ada_randomized_search = RandomizedSearchCV(estimator = AdaBoostClassifier(estimator = DecisionTreeClassifier()), 
                                     param_distributions = ada_param_grid, 
                                     cv = 3, 
                                     scoring = 'f1',
                                     n_jobs = -1, 
                                     n_iter = 10).fit(x_train, y_train)

#extracting best hyperparameter combination
ada_randomized_search.best_params_

{'n_estimators': 300,
 'learning_rate': 0.01,
 'estimator__min_samples_split': 10,
 'estimator__min_samples_leaf': 7,
 'estimator__max_depth': 5}

In [11]:
#building the model with best hyperparameters
ada_md = AdaBoostClassifier(estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 7, max_depth = 5),
                            n_estimators = 300, learning_rate = 0.01).fit(x_train, y_train)

#predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
ada_label = precision_recall_cutoff(y_test, ada_pred)

#print classification report
print(classification_report(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2286
           1       0.95      0.93      0.94       714

    accuracy                           0.97      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000



Based on the results above, the model using RandomizedSearchCV slightly outperforms the model using GridSearchCV